In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch.distributions import Dirichlet, Bernoulli, Uniform
import pandas as pd
import tqdm as tm


from src.model_gen import *
from src.visualize_latent_space import *
from src.Dirichlet_Reg import *
from src.alignment import *


# from sklearn.decomposition import TruncatedSVD as psvd
# import plotly.express as px
# from torch import nn
# from torch.utils.data import DataLoader
# from torchvision import datasets
# from torchvision.transforms import ToTensor
# from numpy import random as rd
# from numpy import arange as seq
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [2]:
#default settings
T, n = 2, 3000
# mat = torch.tensor([[36,2,27], [99,86,14], [72,18,34]], dtype= torch.float).reshape(K, p)/10
mat = [[1, 1, 10], [1, 10, 1], [10, 1, 1]]
model = ABC_sim(time = T,
                nodes = n,
                beta = [1,1,-5,5.0],
                alpha_0 = mat)


In [3]:
Z0 = model.synth_data["lat_pos"][0,]
Z1 = model.synth_data["lat_pos"][1,]

Y0 = model.synth_data["obs_adj"][0,]
Y1 = model.synth_data["obs_adj"][1,]

X0 = ABC_sim.gen_X(Y0, Z0, 3)

In [5]:
temp_est = Dirichlet_GLM_log(predictor = X0, 
                             response = Z1, 
                             constrained = False, 
                             beta_guess = model.settings.beta)
temp_est.est_result

{'final_estimate': tensor([[ 1.1295,  0.0898, -0.8296],
         [-0.0608,  0.9883, -0.9311],
         [ 0.8349, -0.0869, -1.2120],
         [-0.0153,  0.9439, -1.1704],
         [-4.8970,  0.1502,  5.1067],
         [-0.0580, -5.0254,  4.9108],
         [ 5.0065,  4.9585,  2.0248]], device='cuda:0'),
 'final_fisher_info': tensor([[ 27413.6582,  -8698.8564, -18668.2637,   3455.4292,  -1134.4171,
           -2282.5549,  27614.4102,  -8455.9561, -19115.4961,   3320.2271,
           -1355.8324,  -1921.8837,   5212.8452,  -1771.0300,  -3386.4399,
           17637.4922,  -5015.3286, -12581.8047,  34288.2578, -10636.8086,
          -23526.8867],
         [ -8698.8555,  10719.0068,  -1658.9467,  -1134.4172,   3765.3313,
           -2591.7483,  -8455.9570,  10045.8418,  -1229.7994,  -1355.8324,
            4437.4429,  -3041.2393,  -1771.0297,   5921.6938,  -4070.7158,
           -5015.3286,   6751.5659,  -1504.0342, -10636.8076,  18002.0605,
           -6903.1294],
         [-18668.2656,  -165

In [6]:
ora_est_0 = Oracle_Align(Z0, Y0, 2)
ora_est_1 = Oracle_Align(Z1, Y1, 2)

In [9]:
X0_tilde = ABC_sim.gen_X(Y0, ora_est_0.embed_aligned, 3)
Z1_tilde = ora_est_1.embed_aligned
ora_reg = Dirichlet_GLM_log(predictor = X0_tilde, 
                            response = Z1_tilde,
                            constrained = True,
                            beta_guess = model.settings.beta)

In [10]:
ora_reg.est_result

{'final_estimate': tensor([[ 0.9630,  0.0000, -0.9630],
         [ 0.0000,  0.9630, -0.9630],
         [ 1.0566,  0.0000, -1.0566],
         [ 0.0000,  1.0566, -1.0566],
         [-4.9545,  0.0000,  4.9545],
         [ 0.0000, -4.9545,  4.9545],
         [ 4.1272,  4.1272,  1.1923]], device='cuda:0'),
 'final_fisher_info': tensor([[30795.0742, 28723.4922, -9450.5020,   358.7685],
         [28723.4922, 28411.1992, -9247.7754,   371.0063],
         [-9450.5020, -9247.7754,  4675.5527,  1309.9995],
         [  358.7684,   371.0065,  1309.9993,  2992.5859]]),
 'ASE_info_lost': 0.051000000000000045}